In [25]:
import pandas as pd
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Ensure NLTK dependencies are available
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sivaj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sivaj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [63]:
# Load the dataset
data = pd.read_csv("D:/AI Engineer/sentiment_analysis.csv")  # Replace with the actual file path

data

,Year,Month,Day,Time of Tweet,text,sentiment,Platform
0,2018,8,18,morning,What a great day!!! Looks like dream.,positive,Twitter
1,2018,8,18,noon,"I feel sorry, I miss you here in the sea beach",positive,Facebook
2,2017,8,18,night,Don't angry me,negative,Facebook
3,2022,6,8,morning,We attend in the class just for listening teac...,negative,Facebook
4,2022,6,8,noon,"Those who want to go, let them go",negative,Instagram
...,...,...,...,...,...,...,...
494,2015,10,18,night,"According to , a quarter of families under six...",negative,Twitter
495,2021,2,25,morning,the plan to not spend money is not going well,negative,Instagram
496,2022,5,30,noon,uploading all my bamboozle pictures of facebook,neutral,Facebook
497,2018,8,10,night,congratulations ! you guys finish a month ear...,positive,Twitter


In [65]:
# Keep only the 'text' and 'sentiment' columns
data = data[['text', 'sentiment']]

# Check for null values and drop rows with missing text or sentiment
data.dropna(subset=['text', 'sentiment'], inplace=True)

C:\Users\sivaj\AppData\Local\Temp\ipykernel_26404\3139721180.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(subset=['text', 'sentiment'], inplace=True)


In [67]:
# Preprocessing function
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters (non-alphabetical)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize the text
    words = word_tokenize(text)
    # Remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]
    return ' '.join(words)

# Apply preprocessing to the 'text' column
data['clean_text'] = data['text'].apply(preprocess_text)


C:\Users\sivaj\AppData\Local\Temp\ipykernel_26404\1448965178.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['clean_text'] = data['text'].apply(preprocess_text)


In [69]:
# Convert sentiment labels to numerical values
sentiment_mapping = {'positive': 1, 'negative': 0, 'neutral': 2}
data['sentiment_label'] = data['sentiment'].map(sentiment_mapping)


C:\Users\sivaj\AppData\Local\Temp\ipykernel_26404\3022189017.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentiment_label'] = data['sentiment'].map(sentiment_mapping)


In [71]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['clean_text'], data['sentiment_label'], test_size=0.2, random_state=42)


In [73]:
# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [75]:
# Train a Logistic Regression model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)


LogisticRegression()

In [77]:
# Predictions
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:\n', classification_report(y_test, y_pred))


Accuracy: 0.63
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.47      0.61        36
           1       0.80      0.59      0.68        34
           2       0.47      0.87      0.61        30

    accuracy                           0.63       100
   macro avg       0.71      0.64      0.63       100
weighted avg       0.72      0.63      0.63       100



In [79]:
# Example prediction function
def predict_sentiment(text):
    processed_text = preprocess_text(text)
    vectorized_text = vectorizer.transform([processed_text])
    prediction = model.predict(vectorized_text)[0]
    sentiment_labels = {1: 'positive', 0: 'negative', 2: 'neutral'}
    return sentiment_labels[prediction]

# Test the function with a sample text
sample_text = "I love this weather!"
print(f'Sentiment of sample text: {predict_sentiment(sample_text)}')


Sentiment of sample text: positive


In [81]:
sample_text = "The service was awful, and the food was cold. I will never come back!"
print(f'Sentiment of sample text: {predict_sentiment(sample_text)}')


Sentiment of sample text: neutral
